# Lecture 2: Functions and procedural abstraction, FP [draft]

* Functions
* Some brief notes on style, debugging and testing.
* Declarative patterns and functional programming.
* Presentation by Anders Märak Leffler
* Attribution: extends work by Johan Falkenjack.
* License: [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

# Bonus information

* Early [preview of Python 3.8](https://www.python.org/downloads/release/python-380a1/) available. (Featuring [walrus operator](https://medium.com/hultner/try-out-walrus-operator-in-python-3-8-d030ce0ce601), apparently).

## Why?

In [ ]:
# A totally made up script, that goes a little overboard (for demonstration purposes).


# First we read the data
with ... :
    # some lines
    
# Then remove all the duplicates
# ...

# Then extract the relevant parameters by some algorithm.

# ... many lines of code cut and pasted ...
# ... many lines of code cut and pasted ...
# ... many lines of code cut and pasted ...
# ... many lines of code cut and pasted ...
# ... many lines of code cut and pasted ...


# Make sure that model_is_consistent is True iff the model is consistent (with some property).

# ... many lines of code cut and pasted ...
# ... many lines of code cut and pasted ...


if model_is_consistent:
    # Lots of code for plotting.
    # (without _any_ procedural abstraction this would be insane!)

In [1]:
# Example, cleaned up.

def run_simulation(fname):
    """Generate a model of data in file fname, and plot it if consistent."""
    data = read_data(fname) 
    remove_duplicates(data) 
    model = extracted_parameters(data) 
    if model.is_consistent():
        plot(data, model)

def read_data(fname):
    """Return the data contents of file named fname."""
    pass

def remove_duplicates(dataset):
    """Remove duplicates from mutable input dataset."""
    pass

def extracted_parameters(dataset):
    """Return a model based on the dataset, with features XYZ"""
    pass

* Structure and procedural abstraction. 
    * Designing programs ("here I need *something that reads the data* and *something that cleans up the input*,..."), rather than "first I do instructions $s_0,\ldots, s_{1000}$ then jump back to step $s_{349}$ if $x$ is even...").
    * Readability.
    * Encapsulating behaviour.
    * Modularity in testing. Did everything fail because of a mistake in the "remove duplicates" chunk of code? We can test that more easily now.

* Practical scripting issues: 
    * less repeated code (less "did I remember to fix the bug in *all* of the copy-pasted code?").
    * redo this calculation, but with other inputs.
* ... and much more ...

Caveats:
* Sometimes having only a simple script makes sense.
* Sometimes replacing function calls with their code makes *enough* sense from a performance perspective. 

## Functions in Python

* Typically defined with `def` or a `lambda`.
* First-class (callable) values. *A function is just an object.*
    * Corollary: you can't have two distinct f(x) and f(x,y) [without tricks].
* Called by `function_name(<arguments>)`.

In [ ]:
def hungarian_method():
    print("My hovercraft is full of eels.")

def print_args(x):
    print("I was called with: ", x)
    return x

In [ ]:
hungarian_method()

In [ ]:
print_args(99)

In [ ]:
# What will this do? Crash? Print? Return?
print_args(hungarian_method)

In [1]:
print_args(hungarian_method{})

SyntaxError: invalid syntax (<ipython-input-1-b0f232e62fbc>, line 1)

* Lambdas contain a single expression, and have implicit return.

In [ ]:
# Single argument lambda
sq = None 

# Two argument lambda
add = None

# No argument lambda
give_me_five = None

In [ ]:
bad_lambda = lambda x : ????

(Yes you can get around this, even without wrapping your code in a function, but you shouldn't. Use a `def` instead in that case.)

In [ ]:
# Optional task: figure out how to make a lambda that imports the math module, requests an input and 
# prints the square root of that input when run.

# This is really bad style, but might be fun to think about in order to understand the language. 
# This also suggests another reason as to why your code shouldn't eval(...) user input willy-nilly, as some textbooks
# suggest.

* When are arguments evaluated? Applicative-order evaluation.

In [ ]:
# What will this print?

def f(n):
    print("--- Running f now!")
    return n
    
print_args(print_args(f(100)))

In [ ]:
# What happens to unused parameters?

def test(use_2nd_arg, arg):
    if use_2nd_arg:
        print(arg)
    else:
        print("--- Didn't really need the value.")
        
test(False, f(99))

* Keyword arguments. Useful to clarify calls!

You can combine keyword arguments and regular positional, but positional come first.

[Note: Core developer Raymond Hettinger argues for (over)using this in [PyCon 2013: Transforming Code into Beautiful, Idiomatic Python](https://youtu.be/OSGv2VnC0go?t=1841)]

* Default arguments. Should follow non-default arguments.

In [ ]:
def test(use_2nd_arg=True, arg=5):
    if use_2nd_arg:
        print(arg)
    else:
        print("--- Didn't really need the value.")

Helps with understanding [the documentation](https://docs.python.org/3.7/library/functions.html#print).

In [ ]:
# What will happen here?

def add_to_list(e, lst=[]):
    lst.append(e)
    return lst

my_list = add_to_list(999)
add_to_list(123, my_list)
print(my_list)
my_other_list = add_to_list("bat")
add_to_list("man", my_other_list)
print(my_other_list)

* Functions can be defined within functions.
* Static/lexical scoping. What will the following print?

In [ ]:
# What will this print?

x = 5
def f():
    print("I'm in f! Now x is ", x)
    
def g(x):
    print("I'm in g! Now x is ", x)
    f()

print("--- Calling f.")
f()

print("--- Calling g (which in turn calls f).")
g(99)

* LEGB lookup (first Local, then Enclosing, Global and finally Builtin scope).
* Binds values locally (unless told otherwise).

In [ ]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

In [ ]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    x = 5
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

In [ ]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    x = x + 1
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

In [ ]:
# Exercise: play around with the keywords nonlocal and global.

   * Function + their scope/context (the latter word used losely) form a **closure** (or "lexical closure").

In [149]:
x = 1
y = 2

def f(x):
    def g():
        print("In this g, x is", x, " and y is ", y)
    return g

f(100)

In [147]:
# Note: the lambda doesn't get x0 or x1 by parameter. It has to get them from somewhere!

def points_close_to(points, x0, y0, max_dist):
    is_near = lambda coord : (coord[0] - x0)**2 + (coord[1] - y0)**2 <= max_dist**2
    return list(filter(is_near, points))

points = [ (1,2), (5,9), (3,3), (2, 5) ]
points_close_to(points, 3,3, 3)

[(1, 2), (3, 3), (2, 5)]

In [170]:
# Bonus exercise (way outside the scope of this course): can we reach into a closure? To read values? Write?
# Python is flexible with certain things, and others not so much.

h100 = f(100)   
print(h100)
# Play around with h100. Check the dunder methods. (those that start and end with __)

<function f.<locals>.g at 0x7f1758418598>


[//]: # (Functions are just another type of value. Nothing special.)

# A note on naming, style, testing

---
** YOU DO NOT WRITE CODE FOR THE INTERPRETER. YOU WRITE IT FOR OTHER HUMAN READERS. **

---

* You should understand your code.
* Others should understand your code (and convince themselves of its correctness).
* ...one of those 'others' is future you.

### Python function style interlude

> Names that are visible to the user as public parts of the API should follow conventions that reflect usage rather than implementation.

The "Overriding Principle" from the official [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/#overriding-principle).

**In other words: focus on ** ***what*** **it does (or return), not** ***how*** **it does it.**

In [ ]:
# Slightly obfuscated. What's a good name?

def f(x):
    for y in x:
        x[y] += 1

Consider
* function name.
* parameter names.
* variables.

In [ ]:
help(max)

In [ ]:
# Adding this to the function above.

**Not** required in labs, but useful. Official guidelines: [PEP 257 -- Docstring Conventions](https://www.python.org/dev/peps/pep-0257/).

## A few general hints for design and naming

Caveat: large subject, involving personal preferences and official guidelines (sometimes at company level).

* Struggling with naming can mean that a function does too many different things.
    * Separate effects (print this, write file, change data by deaveraging...) from computation (is this valid data? What is the mean of this?).
* Take the **calling function's perspective** when naming.
    * Ex: When writing a consistency-checking function, consider if `if is_consistent(data) ...` would more sense than `if calculate_if_data_consistent(data) ...` *for the function calling the one you're writing*.
* **Don't mislead**. `is_consistent(data)` should test if it is consistent, not change the data, print something or the like.

* **Consider the application!** What does the name represent? (Cf last lecture.)

In [ ]:
# Example
names = ["Alonzo", "Zeno", "Trisse"]

# Bad. j sounds like an index. (i,j,k...)
for i, j in enumerate(names):
    print("The {}th name is {}.".format(i,j))

In [ ]:
# OK, we know that it's a list of strings. But the strings _represent_ something.
for i, string in enumerate(names):
    print("The {}th name is {}.".format(i,string))

In [ ]:
for i, name in enumerate(names):
    print("The {}th name is {}.".format(i,name))

* In *small-scale* top-down design, creating "empty" functions with a certain contract can be helpful.
    * Example: "In my program I will need a function `read_data` which takes a file name for a CSV file and returns a data set on format *X*." (this is the contract) "I'll define it, and fill it in later."
    * A Python function body cannot be empty. `pass` is your friend.

## A note on testing and (unsystematic) debugging

> It has been just so in all of my inventions. The first step is an intuition, and comes with a burst, then difficulties arise - this thing gives out and [it is] then that 'Bugs' - as such little faults and difficulties are called - show themselves and months of intense watching, study and labor are requisite.

*Thomas Edison*, quoted in Ammann & Offutt (2017) - Intro to Software Testing, 2nd ed.

* Plenty of tools out there for debugging.
    * One example: [pdb](https://docs.python.org/3.7/library/pdb.html).
* We will focus on simpler print-style and asserts.

In [ ]:
# Faulty code. Somewhat articificial example. 
# Also an inefficient method, but let's not worry about that.

def calculate_if_prime_or_not(x):
    # Check to see if we can find something that divides the number.
    for y in range(x):
        if y % x == 0:
            answer = False
        else:
            answer = True
    return answer

What are the issues? Found how?

True

In [ ]:
def update_result(result, i):
    result = result + i

def sum_values(n):
    result = 0
    for i in range(n):
        update_result(result, i)
    return result

In [220]:
# Bonus segment: an obvious programming way to make it faster (without square roots etc)?

# Declarative, 
def is_prime_length_based(n):
    return len([d for d in range(2,n) if n % d == 0]) == 0
    # could be written not len(...)
    
def is_prime_breaking(n):
    return None not in (None for d in range(2,n) if not n % d)

# What happens if we change () into []?

import profile
N = 99999999
print("--- is_prime")
profile.run("is_prime({})".format(N))

print("--- is_prime_breaking")
profile.run("is_prime_breaking({})".format(N))

print("--- is_prime_length_based")
profile.run("is_prime_length_based({})".format(N))

True
True
False
True
--- is_prime
         7 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 <ipython-input-53-46afc67f094d>:3(is_prime)
        2    0.000    0.000    0.000    0.000 <ipython-input-53-46afc67f094d>:4(<genexpr>)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 profile:0(is_prime(99999999))
        0    0.000             0.000          profile:0(profiler)


--- is_prime_breaking
         7 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000 

* We can add assertions to show that our (pure) function passed some useful tests. Corner cases to illustrate?

In [ ]:
assert is_prime(???) == ???, "???"

[Note: We might have other forms of properties that might be suitable for automatic testing. For instance "all return values should be True/False", "all multiples of two integers should be non-prime" or the like. This is outside the  scope of this course, but an interesting subject to look into.

Modest focus here: strategies to find good test cases to convince ourselves somewhat.]



* Some of these can be expressed as input-output pairs. How to use?

In [ ]:
def test_restricted_sqrt():
    pass 
    print("--- test_restricted_sqrt succeeded")

[Note: we can test functions which depend on state as well. We need to set up (relevant) state, run tests and tear it down afterwards. You might want to look at eg [unittest](https://docs.python.org/3.7/library/unittest.html) or other frameworks if you are interested in this.]

# Paradigms

* Philosophies about code.
    * Execution model.
    * Code organization.
* Python is a multi-paradigm language (sort of).

# Functional programming

* Typically declarative.
* Purity (see below) makes it easy to reason about. (Not necessarily write...)
    * In particular for parallel computation.
    * MapReduce.

### Pure functions

* No side causes. 
    * Doesn't depend on the state (of the world).

In [196]:
import datetime

def has_side_causes():
    return datetime.datetime.now()

def no_side_causes():
    return 5 + 5
    
has_side_causes()

datetime.datetime(2019, 2, 7, 16, 46, 6, 971550)

* No side effects.
    * Doesn't change the state of the world.

Corollary: In (purely) functional programming, 
* there will be less of hidden dependencies on state.
* reduced complexity.
* referential transparency. Calling function $f$ with input $X$ will always yield the same result.
* mathematical resoning useful (not just "do X, then Y").

## Feature: higher order functions

* Functions with functions as outputs are generally considered higher-order. Have you seen those before? (Functions + scope.)

In [2]:
# Exercise.

* Similarly with functions taking functions as input, or returning functions as output.

In [202]:
# Implementing our own filter, with a list comprehension.

def points_satisfying(points, to_keep):
    """Return a list of points which satisfy the criterion set out by the to_keep function."""
    return [p for p in points if to_keep(p)]

points = [ (1,2), (-2,3), (5,100) , (0,-1) ]
points_satisfying(points, lambda p : p[0] >= 0 and p[1] >= 0)

[(1, 2), (5, 100)]

## Declarative patterns

* Many declarative-style patterns are Pythonic.
* Often implemented with iterators.
* Useful to chain together, one result feeding into the next.
* Often well-implemented.
* Example: map.

In [226]:
# map is a standard functional programming pattern.
# Cf mathematical mapping. map takes a sequence of values and produces the image of that sequence under a given function.

print(map(lambda x : x**2, [1,2,3]))
tuple(map(lambda x : x**2, [1,2,3]))

(1, 4, 9)

In [227]:
# Caveat: in Python this produces something which generates the values as you ask for them.

squares = map(lambda x : x**2, [1,2,3])
print(squares)
print(list(squares))   # Consumes all the values.
print(tuple(squares))  # All consumed.

[1, 4, 9]
()


Corollary: this can be quite efficient. Doesn't generate values in vain.

* Example: filter

In [3]:
# filter keeps those values which conform to a predicate (function which - ideally - returns True/False depending
# on if a criterion has been satisfied).

is_even = lambda n : n % 2 == 0       # could be written "not n % 2"
filter(is_even, [1,2,3])
list(filter(is_even, [1,2,3]))

[2]

* Example reduce. 

In [232]:
from functools import reduce

reduce(lambda acc, e : [acc] + [e], [1,2,3, 4])   # [  [ [1,2] ] + [3]  ] + [4]

[[[1, 2], 3], 4]

* These can be combined.

In [ ]:
sq = lambda x : x**2
seq = range(100)

square_evens = ???

## A note on recursion

* Practiced in lab 2B. 
* Function values defined in terms... of function values.
* Focus on mathematical identities.
* Sometimes useful in its own right.
* Sometimes inefficient, but useful when turned into non-recursive algorithms.
* Use cases here: many tree-structures.

* Strategy
    * "Pretend" that we know the solution to a smaller case, extend that.
    * What do we know about the *smallest* case (or the smallest cases)?

In [4]:
# What is the sum 0 + 1 + ... + (n-1) + n?

# Helper function (during program construction).
def sum_reference(n):
    """Return the sum 0+...+n."""
    return sum(range(n+1))

# If we knew the sum 0 + 1 + ... + (n-1)?

def sum_rec(n):
    pass

* Handwaving explanation above. Good mathematical foundations. 
    * Closely related to inductive proofs.
    * Well-orderings.

* Sanity checks:
    * base case (or base cases). 
    No more subdivision of the problem.
    Typical examples: empty list, zero, reached the leaves in a decision tree.
    * recursive case. Split into smaller case, and write the solution to this case. (Typically: rest of the list, traverse left or/and right branches of the tree...)